<a href="https://colab.research.google.com/github/SRT2207/Structured_SSM_for_EHR_Classification_Group38/blob/main/dataload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import json
import pandas as pd
import tarfile
import sys

## Load data and package requirements


In [3]:
# Install the required packages
!pip install -r Structured_SSM_for_EHR_Classification_Group38/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118, https://data.pyg.org/whl/torch-2.2.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.7/811.7 MB 812.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 71.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 69.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subproc

In [12]:
# Define the path to the P12data directory
data_path = 'Structured_SSM_for_EHR_Classification_Group38/P12data/'

# List all .json files in the directory
json_files = [f for f in os.listdir(data_path) if f.endswith('.json')]

# Load each JSON file into a DataFrame with the same name as the file
json_df = {}
for file in json_files:
    file_path = os.path.join(data_path, file)
    with open(file_path, 'r') as f:
        data = json.load(f)
        # Normalize JSON data into a DataFrame
        df = pd.json_normalize(data)
        # Store in a dictionary with the key as the filename
        json_df[file.replace('.json', '')] = df

# Display the keys of the dictionary
print("DataFrames created:", list(json_df.keys()))

DataFrames created: ['normalization_physionet2012_5', 'normalization_physionet2012_3', 'normalization_physionet2012_2', 'normalization_physionet2012_1', 'normalization_physionet2012_4']


In [5]:
# Dictionary to store DataFrames with the name of the .npy file as the key
dataframes = {}

# List all .tar.gz files in the directory
tar_files = [f for f in os.listdir(data_path) if f.endswith('.tar.gz')]

# Loop through each .tar.gz file, extract, and load .npy data into DataFrames
for tar_file in tar_files:
    tar_path = os.path.join(data_path, tar_file)
    extract_folder = os.path.join(data_path, tar_file.replace('.tar.gz', ''))

    # Create directory for extracted files
    os.makedirs(extract_folder, exist_ok=True)

    # Extract the tar.gz file
    with tarfile.open(tar_path, 'r:gz') as tar:
        tar.extractall(path=extract_folder)

    # Load each .npy file in the extracted folder
    for root, _, files in os.walk(extract_folder):
        for file in files:
            file_path = os.path.join(root, file)

            # Check if the file is a .npy file
            if file.endswith('.npy'):
                # Load the .npy file with allow_pickle=True
                array_data = np.load(file_path, allow_pickle=True)

                # Convert to DataFrame
                df = pd.DataFrame(array_data)

                # Store the DataFrame in the dictionary using the .npy file's name (without extension) as the key
                file_name_without_extension = file.replace('.npy', '')
                dataframes[file_name_without_extension] = df

# Display the keys of the dictionary to verify the DataFrames are named correctly
print("DataFrames created:", list(dataframes.keys()))


DataFrames created: ['test_physionet2012_1', 'train_physionet2012_1', 'validation_physionet2012_1', 'train_physionet2012_3', 'validation_physionet2012_3', 'test_physionet2012_3', 'validation_physionet2012_5', 'test_physionet2012_5', 'train_physionet2012_5', 'train_physionet2012_2', 'validation_physionet2012_2', 'test_physionet2012_2', 'validation_physionet2012_4', 'test_physionet2012_4', 'train_physionet2012_4']


In [11]:
# for name, df in dataframes.items():
#     print(f"Contents of DataFrame '{name}':")
#     display(df.head())


## Load models

In [8]:
# Define the path to the models directory
models_path = 'Structured_SSM_for_EHR_Classification_Group38/models'

# Add the models directory to the system path
sys.path.append(models_path)

In [10]:
import grud  # Import the module

# List all functions and classes in grud model
print(dir(grud))

['F', 'GRUD', 'GRUDCell', 'GRUDModel', 'List', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'exp_relu', 'generate_masks', 'get_activation', 'jit', 'masked_max_pooling', 'masked_mean_pooling', 'nn', 'torch']
